In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [4]:
df = pd.read_csv("Train_Data.csv")
test=pd.read_csv("Test_Data.csv")

In [5]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
le = LabelEncoder()
# lb = LabelBinarizer()
df['adgroup'] = le.fit_transform(df['adgroup'])
test['adgroup']=le.transform(test['adgroup'])
# df = df.join(pd.DataFrame(lb.fit_transform(df['adgroup'])))
# test = test.join(pd.DataFrame(lb.transform(test['adgroup'])))
# df.drop(['adgroup'],axis=1)
# test.drop(['adgroup'],axis=1)
df = df.set_index(df['date'])
test = test.set_index(test['date'])

df.drop(['date', 'campaign'],axis=1,inplace=True)
test.drop(['date','campaign'],axis=1,inplace=True)


In [6]:
for i in range(len(df)):
    df['ad'][i] = int(df['ad'][i][3:])
for i in range(len(test)):
    test['ad'][i]=int(test['ad'][i][3:])
df['ad']=pd.to_numeric(df['ad'])
test['ad']=pd.to_numeric(test['ad'])
df.drop(['ad'],axis=1,inplace=True)

<ipython-input-6-f954e414b656>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ad'][i] = int(df['ad'][i][3:])
<ipython-input-6-f954e414b656>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ad'][i]=int(test['ad'][i][3:])


In [7]:
df['CTR']=df['clicks']/df['impressions']
df['CPC']=df['cost']/df['clicks']
df['CPA']=df['cost']/df['conversions']

test['CTR']=test['clicks']/test['impressions']
test['CPC']=test['cost']/test['clicks']
test['CPA']=test['cost']/test['conversions']

df['CPC'].fillna(0,inplace=True)
df['CPA'].fillna(0,inplace=True)

test['CPC'].fillna(0,inplace=True)
test['CPA'].fillna(0,inplace=True)

df.replace([np.inf], 65,inplace=True)
test.replace([np.inf], 65,inplace=True)

In [8]:
# y = df['revenue']
# df.drop(['revenue','campaign'], axis=1,inplace=True)
# test=np.array(test)
# df = np.array(df)

In [9]:
# df = np.array(df)
x_train, x_test = train_test_split(df)

In [8]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [9]:
from tensorflow.keras import backend
import tensorflow as tf
 
def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [10]:
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
model = Sequential()                # could make units 50, for better prediction
# model.add(Dense(units=40,activation='selu'))
model.add(LSTM(units=40,activation='tanh',input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.4))
model.add(Dense(units=40,activation='selu'))
model.add(Dropout(0.4))
# model.add(LSTM(units=40,activation='tanh'))
# model.add(Dense(units=30,activation='relu'))
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=40,activation='selu'))
model.add(Dropout(0.4))
model.add(Dense(units=40,activation='selu'))

model.add(Dense(1,activation='relu'))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mean_squared_error',metrics=[rmse])
model.fit(x_train,y_train, epochs=30, batch_size=60)

NameError: name 'x_train' is not defined

In [ ]:
# m1 = RandomForestRegressor()
# m1.fit(x_train,y_train)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)
y_pred = model.predict(x_test)
rms = mean_squared_error(y_test, y_pred, squared=False)
rms

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
from xgboost import XGBRFRegressor, XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from pycaret.regression import *

In [17]:
g = df.groupby('adgroup')

# model0 = XGBRFRegressor(n_estimators=10,max_depth=6, min_child_weight=2,subsample=0,learning_rate = 0.3)
# model0 = lg(kernel='rbf', degree=2, gamma='auto', C = 0.1)
j=6
model0 = LGBMRegressor(n_estimators=10000,max_depth=4, num_leaves=2**j,boosting_type='dart',learning_rate=0.09,random_state=0)
model1 = LGBMRegressor(random_state=0)
model2 = LGBMRegressor(n_estimators=1000,max_depth=4, num_leaves=2**5,boosting_type='dart',learning_rate=0.8,random_state=0)
model3 = LGBMRegressor(random_state=0)
for i in range(4):
    d=pd.DataFrame(g.get_group(i))
    # y=d['revenue']
    # print(d.head(5))
    # d.drop(['revenue'],axis=1,inplace=True)
    data = setup(d,target = 'revenue', use_gpu=True)
    xgb = create_model('xgboost',verbose=False)
    tuned_xgb = tune_model(xgb, n_iter = 50,optimize='rmse')
    print(evaluate_model(xgb))
    # d_train,d_test,y1_train,y1_test = train_test_split(d,y,random_state=0)
    # if(i==0):
    #     model0.fit(d_train,y1_train)
    #     y_pred=model0.predict(d_test)
        
    # elif (i==1):
    #     model1.fit(d_train,y1_train)
    #     y_pred=model1.predict(d_test)
    # elif (i==2):
    #     model2.fit(d_train,y1_train)
    #     y_pred=model2.predict(d_test)
    # elif (i==3):
    #     model3.fit(d_train,y1_train) 
    #     y_pred=model3.predict(d_test)
    
    # for i in range(len(d_test)):
    #     if(y_pred[i]<0):
    #         y_pred[i] = 0
    # rms = mean_squared_error(y1_test, y_pred, squared=False)
    # print(rms)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,140.7690,114070.7500,337.7436,0.8670,2.0352,1.3581
1,78.2386,29910.6621,172.9470,0.9901,1.5867,0.7374
2,129.3504,102231.2969,319.7363,0.9471,1.7748,1.9018
3,148.7488,140984.2656,375.4787,0.9681,1.5753,0.7336
4,196.2488,309964.2812,556.7444,0.9218,1.7056,0.6306
5,195.4596,233639.1250,483.3623,0.8622,1.7756,0.7022
6,128.7636,78142.4297,279.5397,0.8818,1.6547,0.7862
7,158.9561,154639.8906,393.2428,0.9635,1.9204,0.6141
8,111.3791,58916.4727,242.7272,0.8652,1.4471,0.9506
9,189.5181,203519.7969,451.1317,0.8431,2.1195,0.4910


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

None


IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:17:22
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


In [ ]:
y_pred=[]
# x_test.drop('revenue',axis=1,inplace=True)
for i in range(len(x_test)):
    if(x_test['adgroup'][i]==0):
        y_pred.append(model0.predict(pd.DataFrame(x_test.iloc[i]).transpose()))
    # elif (x_test['adgroup'][i]==1):
    #     y_pred.append(model1.predict(pd.DataFrame(x_test.iloc[i]).transpose()))
    # elif (x_test['adgroup'][i]==2):
    #     y_pred.append(model2.predict(pd.DataFrame(x_test.iloc[i]).transpose()))
    # elif (x_test['adgroup'][i]==3):
    #     y_pred.append(model3.predict(pd.DataFrame(x_test.iloc[i]).transpose()))

y1=[]


rms = mean_squared_error(y1, y_pred, squared=False)
rms

In [ ]:
np.array(x_test.iloc[i,:]).reshape(1,-1)

In [ ]:
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [10],
        'min_child_weight': [ 5],
        # 'subsample': [0.5, 0.7],
        # 'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [200],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [11]:
y_pred[0]

NameError: name 'y_pred' is not defined

In [ ]:
## Final
model2.fit(df,y)
y_final = model2.predict(test)
for i in range(len(y_final)):
    if y_final[i] < 0:
        y_final[i]=0

In [ ]:
pd.DataFrame({'revenue':y_final}).to_csv("submission.csv")

In [ ]:
sns.scatterplot(df['adgroup'],df['revenue'])

In [9]:
from pycaret.regression import *    

In [14]:
data = setup(df,target = 'revenue', use_gpu=True)

,Description,Value
0,session_id,1142
1,Target,revenue
2,Original Data,"(4571, 9)"
3,Missing Values,False
4,Numeric Features,7
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3199, 11)"


In [24]:
models(internal=True)[['Name', 'GPU Enabled']]

,Name,GPU Enabled
ID,,
lr,Linear Regression,False
lasso,Lasso Regression,False
ridge,Ridge Regression,False
en,Elastic Net,False
lar,Least Angle Regression,False
llar,Lasso Least Angle Regression,False
omp,Orthogonal Matching Pursuit,False
br,Bayesian Ridge,False
ard,Automatic Relevance Determination,False


In [32]:
xfb = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,77.5432,50171.5528,223.9901,0.9457,1.2543,0.6371
1,86.2675,163287.0492,404.0879,0.9001,1.2915,0.6954
2,85.1299,90993.1838,301.6508,0.9261,1.2338,0.6373
3,95.2183,217134.5424,465.9770,0.8858,1.3366,0.6777
4,81.6441,140692.9322,375.0906,0.9008,1.2436,0.7876
5,62.7745,23421.7041,153.0415,0.8990,1.3307,1.0753
6,66.1703,32225.5696,179.5148,0.9445,1.1702,0.5825
7,77.7753,128279.7562,358.1616,0.6882,1.1892,0.6662
8,54.3070,20115.3575,141.8286,0.8841,1.2415,1.2727
9,49.0903,17513.4719,132.3385,0.8758,1.2189,0.5610


In [33]:
tuned_xg = tune_model(xfb)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,74.8738,38991.6308,197.4630,0.9578,1.7859,0.6605
1,95.7183,272824.5169,522.3261,0.8331,1.9056,0.6588
2,103.3760,189264.3071,435.0452,0.8463,1.7076,0.6303
3,101.3509,295116.4125,543.2462,0.8448,1.9695,0.6436
4,87.9456,195151.6877,441.7598,0.8624,1.6292,0.8915
5,59.8740,18802.1396,137.1209,0.9190,1.6412,1.1802
6,73.4581,35440.7915,188.2572,0.9389,1.6213,0.7115
7,77.1427,120743.7951,347.4821,0.7065,1.7255,0.6760
8,56.4414,20770.2279,144.1188,0.8804,1.6012,1.4300
9,51.3003,15586.7901,124.8471,0.8894,1.5621,0.5600


In [34]:
finalize_model(xfb)

In [19]:
yf = predict_model(xfb,test)

In [22]:
pd.DataFrame({'revenue':yf['Label']}).to_csv("submission2.csv")

In [36]:
import tensorflow as tf 
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]